In [ ]:
import pandas as pd
import numpy as np
from torchtext.vocab import GloVe
import time
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import preprocessing
import matplotlib.pyplot as plt
import syllables

#### Loads GloVe embeddings with default settings

In [ ]:
df = pd.read_csv('datasets/lcp_single_train.tsv', delimiter='\t')

#Change GloVe cache to GloVe Installation path
glove = GloVe(cache='GloVe/')

print("Loading GloVe embeddings...")
a = time.time()
df['glove'] = df.apply(lambda row: glove[row['token']].numpy(), axis=1)
b = time.time()
print("Finished loading GloVe embeddings in", b-a, "seconds.")

training_data = np.stack(df['glove'].tolist())

df_trial = pd.read_csv('datasets/lcp_single_trial.tsv', delimiter='\t')
df_trial['glove'] = df_trial.apply(lambda row: glove[row['token']].numpy(), axis=1)
trial_data = np.stack(df_trial['glove'].tolist())

#### Defines Plot Building Function

In [ ]:
def plotCharts(predictions, truth):
    plt.plot(predictions, truth, 'o')
    plt.title("Predictions vs. truth on the trial set")
    plt.savefig("trial_regression.png")
    plt.show()

    plt.plot(predictions, predictions - truth, 'o')
    plt.title("Predicted vs L1 error no abs")
    plt.show()

    plt.plot(trial_complexity, predictions - truth, 'o')
    std = np.std(predictions - trial_complexity)
    plt.title("Truth vs L1 error no abs, STD: {}".format(round(std, 4)))
    plt.show()

#### Default Linear Regression

In [ ]:
'''
Train R^2 0.6116870249132482
Train L1: 0.06484457413081363
Train L2: 0.006877313401841914
Trial R^2: 0.6188507127442916
Trial L1: 0.07193707302694131
Trial L2: 0.008524886396647887
'''

linreg = LinearRegression()
complexity = df['complexity']
linreg.fit(training_data, complexity)

predictions = linreg.predict(training_data)
print("Train R^2", linreg.score(training_data, complexity))
print("Train L1:", np.mean(np.abs(predictions - complexity)))
print("Train L2:", np.mean(np.square(predictions - complexity)))

# eval on trial data
trial_predictions = linreg.predict(trial_data)
trial_complexity = df_trial['complexity']

print("Trial R^2:", linreg.score(trial_data, trial_complexity))
print("Trial L1:", np.mean(np.abs(trial_predictions - trial_complexity)))
print("Trial L2:", np.mean(np.square(trial_predictions - trial_complexity)))

plotCharts(trial_predictions, trial_complexity)

# SVR

### Default SVR

In [ ]:
"""
Default SVR
Train R^2 0.7414494251127137
Train L1: 0.056722483705022836
Trial R^2: 0.6400853869204248
Trial L1: 0.06904373273713046
""" 

from sklearn import svm
svrreg = svm.SVR(C=1)
complexity = df['complexity']
svrreg.fit(training_data, complexity)

predictions = svrreg.predict(training_data)
print("Train R^2", svrreg.score(training_data, complexity))
print("Train L1:", np.mean(np.abs(predictions - complexity)))
print("Train L2:", np.mean(np.square(predictions - complexity)))

# eval on trial data
trial_predictions = svrreg.predict(trial_data)
trial_complexity = df_trial['complexity']

print("Trial R^2:", svrreg.score(trial_data, trial_complexity))
print("Trial L1:", np.mean(np.abs(trial_predictions - trial_complexity)))
print("Trial L2:", np.mean(np.square(trial_predictions - trial_complexity)))

plotCharts(trial_predictions, trial_complexity)

### SVR with Epsilon=0.02

In [ ]:
"""
SVR Epsilon = 0.02
Train R^2 0.8299065298168575
Train L1: 0.039219046136949286
Trial R^2: 0.6616577283798268
Trial L1: 0.06703706379369276
""" 

from sklearn import svm
svrreg = svm.SVR(epsilon=.02)
complexity = df['complexity']
svrreg.fit(training_data, complexity)

predictions = svrreg.predict(training_data)
print("Train R^2", svrreg.score(training_data, complexity))
print("Train L1:", np.mean(np.abs(predictions - complexity)))

# eval on trial data
trial_predictions = svrreg.predict(trial_data)
trial_complexity = df_trial['complexity']

print("Trial R^2:", svrreg.score(trial_data, trial_complexity))
print("Trial L1:", np.mean(np.abs(trial_predictions - trial_complexity)))

plotCharts(trial_predictions, trial_complexity)

### SVR with Epsilon=0

In [ ]:
"""
SVR Epsilon = 0.01 With Bagging
Train R^2 0.8171951889110142
Train L1: 0.04270026210792148
Trial R^2: 0.65143687729242
Trial L1: 0.06787063160575084
""" 

from sklearn import svm
from sklearn.ensemble import BaggingRegressor
svrreg = BaggingRegressor(base_estimator=svm.SVR(epsilon=0.01), random_state=0, max_samples=0.5, max_features=0.5)
complexity = df['complexity']
svrreg.fit(training_data, complexity)

predictions = svrreg.predict(training_data)
print("Train R^2", svrreg.score(training_data, complexity))
print("Train L1:", np.mean(np.abs(predictions - complexity)))

# eval on trial data
trial_predictions = svrreg.predict(trial_data)
trial_complexity = df_trial['complexity']

print("Trial R^2:", svrreg.score(trial_data, trial_complexity))
print("Trial L1:", np.mean(np.abs(trial_predictions - trial_complexity)))

plotCharts(trial_predictions, trial_complexity)

In [ ]:
from sklearn import svm
from sklearn.ensemble import BaggingRegressor
svrreg = BaggingRegressor(base_estimator=svm.SVR(epsilon=0.01, C=0.1), random_state=0)
complexity = df['complexity']
svrreg.fit(training_data, complexity)

predictions = svrreg.predict(training_data)
print("Train R^2", svrreg.score(training_data, complexity))
print("Train L1:", np.mean(np.abs(predictions - complexity)))

# eval on trial data
trial_predictions = svrreg.predict(trial_data)
trial_complexity = df_trial['complexity']

print("Trial R^2:", svrreg.score(trial_data, trial_complexity))
print("Trial L1:", np.mean(np.abs(trial_predictions - trial_complexity)))

plotCharts(trial_predictions, trial_complexity)

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators=10, max_depth=1000)
complexity = df['complexity']
reg.fit(training_data, complexity)

predictions = reg.predict(training_data)
print("Train R^2", reg.score(training_data, complexity))
print("Train L1:", np.mean(np.abs(predictions - complexity)))

# eval on trial data
trial_predictions = reg.predict(trial_data)
trial_complexity = df_trial['complexity']

print("Trial R^2:", reg.score(trial_data, trial_complexity))
print("Trial L1:", np.mean(np.abs(trial_predictions - trial_complexity)))

plotCharts(trial_predictions, trial_complexity)

# Syllables

In [ ]:
def createData(df):
    df['glove'] = df.apply(lambda row: glove[row['token']].numpy(), axis=1)
    df['syllables'] = df['token'].apply(lambda x: syllables.estimate(str(x)))
    df['syllables_scaled'] = preprocessing.scale(df['syllables'])
    df['features'] = df.apply(lambda row: np.append(row['glove'], row['syllables_scaled']), axis=1)
    data = np.stack(df['features'].tolist())
    return data

#### Creates Syllable Training Data

In [ ]:
training_data_syllables = createData(df)
trial_data_syllables = createData(df_trial)
print(training_data_syllables)

In [ ]:
linreg = LinearRegression()
complexity = df['complexity']
linreg.fit(training_data_syllables, complexity)

predictions = linreg.predict(training_data_syllables)
print("Train R^2", linreg.score(training_data_syllables, complexity))
print("Train L1:", np.mean(np.abs(predictions - complexity)))

# eval on trial data
trial_predictions = linreg.predict(trial_data_syllables)
trial_complexity = df_trial['complexity']

print("Trial R^2:", linreg.score(trial_data_syllables, trial_complexity))
print("Trial L1:", np.mean(np.abs(trial_predictions - trial_complexity)))

plotCharts(trial_predictions, trial_complexity)

In [ ]:
"""
SVR Epsilon = 0.01
Train R^2 0.8288651784113816
Train L1: 0.0361685926619976
Trial R^2: 0.6639123014282307
Trial L1: 0.0666936906337456
""" 

from sklearn import svm
svrreg = svm.SVR(epsilon=0.01)
complexity = df['complexity']
svrreg.fit(training_data_syllables, complexity)

predictions = linreg.predict(training_data_syllables)
print("Train R^2", linreg.score(training_data_syllables, complexity))
print("Train L1:", np.mean(np.abs(predictions - complexity)))

# eval on trial data
trial_predictions = linreg.predict(trial_data_syllables)
trial_complexity = df_trial['complexity']

print("Trial R^2:", linreg.score(trial_data_syllables, trial_complexity))
print("Trial L1:", np.mean(np.abs(trial_predictions - trial_complexity)))

plotCharts(trial_predictions, trial_complexity)

In [ ]:
"""
SVR Epsilon = 0.01 With Bagging
Train R^2 0.8171951889110142
Train L1: 0.04270026210792148
Trial R^2: 0.6632770023442027
Trial L1: 0.06634198242108581
""" 

from sklearn import svm
from sklearn.ensemble import BaggingRegressor
svrreg = BaggingRegressor(base_estimator=svm.SVR(epsilon=0.01), random_state=0)
complexity = df['complexity']
svrreg.fit(training_data_syllables, complexity)

predictions = linreg.predict(training_data_syllables)
print("Train R^2", linreg.score(training_data_syllables, complexity))
print("Train L1:", np.mean(np.abs(predictions - complexity)))

# eval on trial data
trial_predictions = linreg.predict(trial_data_syllables)
trial_complexity = df_trial['complexity']

print("Trial R^2:", linreg.score(trial_data_syllables, trial_complexity))
print("Trial L1:", np.mean(np.abs(trial_predictions - trial_complexity)))

plotCharts(trial_predictions, trial_complexity)